In [1]:
import os
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
os.mkdir('data')

In [4]:
# image preprocessing
transform = transforms.Compose([transforms.Resize(40), 
                                transforms.RandomHorizontalFlip(), 
                                transforms.RandomCrop(32), 
                                transforms.ToTensor()])

In [5]:
# datasets
train_dsets = dsets.CIFAR10(root='data/', train=True, download=True, transform=transform)
test_dsets = dsets.CIFAR10(root='data/', train=False, transform=transforms.ToTensor())

In [13]:
# dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dsets, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dsets, batch_size=256, shuffle=False)

In [14]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [15]:
# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [16]:
# ResNet Module
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(conv3x3(self.in_channels, out_channels, stride=stride), 
                                       nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [17]:
# model
resnet = ResNet(ResidualBlock, [2,2,2,2])
resnet.cuda()

# loss and optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=learning_rate)

In [18]:
print(resnet)

ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    )
  )
  (layer2): Sequential(
    (0): Re

In [19]:
# Training
epochs = 40
i = 0
for epoch in range(epochs):
    for images, labels in train_loader:
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        i += 1
        if i % 200 == 0:
            print("Epoch: {}/{}...".format(epoch+1, epochs), 
                  "Iterations: {}...".format(i), 
                  "Loss: {}".format(loss.data[0]))
            
    # decaying learning rate
    if (epoch+1) % 20 == 0:
        learning_rate /= 3
        optimizer = torch.optim.Adam(resnet.parameters(), lr=learning_rate)
    

Epoch: 2/40... Iterations: 200... Loss: 1.317552089691162
Epoch: 3/40... Iterations: 400... Loss: 1.056356430053711
Epoch: 4/40... Iterations: 600... Loss: 0.851357638835907
Epoch: 5/40... Iterations: 800... Loss: 0.8477465510368347
Epoch: 6/40... Iterations: 1000... Loss: 0.8650757074356079
Epoch: 7/40... Iterations: 1200... Loss: 0.7446025013923645
Epoch: 8/40... Iterations: 1400... Loss: 0.8066689968109131
Epoch: 9/40... Iterations: 1600... Loss: 0.5961554050445557
Epoch: 10/40... Iterations: 1800... Loss: 0.5839317440986633
Epoch: 11/40... Iterations: 2000... Loss: 0.5513363480567932


KeyboardInterrupt: 

In [ ]:
# Testing
corr = total = 0

for images, labels in test_loader:
    images = Variable(images.cuda())
    outputs = resnet(images)
    _, pred = torch.max(outputs, 1)
    total += labels.size(0)
    corr += (pred.cpu() == labels).sum()
    
print('Accuracy: {}%'.format(100 * corr / total))

In [ ]:
torch.save(resnet.state_dict(), 'resnet.pkl')